In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import PIL
import PIL.Image 
from keras.preprocessing import image
from tensorflow.keras.models import Sequential

In [ ]:
df_train = pd.read_csv('/kaggle/input/kaggledays-flying-circus/train.csv')
df_train.head()

In [ ]:
df_train['celebrity'].unique()

In [ ]:
sub = pd.read_csv('/kaggle/input/kaggledays-flying-circus/sample_submission.csv')
sub.head()

In [ ]:
hobbies = pd.read_csv('/kaggle/input/kaggledays-flying-circus/hobbies.csv')
hobbies.head()

In [ ]:
hobbies['hobbies'].unique()

In [ ]:
hobbies.columns

In [ ]:
df_train['age'].unique()

Splitting the age column from range to seprate cols for low and high values

# Data preprocessing

In [ ]:
hobbies[['age_low','age_high']] = hobbies['age'].str.split('-',expand=True)
hobbies.drop(['age'], axis=1, inplace=True)
hobbies['age_low'] = hobbies['age_low'].astype(int)
hobbies['age_high'] = hobbies['age_high'].astype(int)
hobbies.head()

In [ ]:
hobbies.columns

In [ ]:
df_train.loc[1,'img_path']

# Image preprocessing

In [ ]:
df_train['celebrity'] = df_train['celebrity'].map({'Actress': 0, 'Actor': 1})

In [ ]:
train_image = []
train_labels = []

for i in tqdm(range(df_train.shape[0])):
    img = image.load_img('/kaggle/input/kaggledays-flying-circus/images/images/'+df_train.loc[i,'img_path'], target_size=(28,28,1), 
                grayscale=True)

    img = image.img_to_array(img)

    img = img/255

    train_image.append(img)
    train_labels.append([df_train.loc[i,'celebrity'], df_train.loc[i,'age']])


In [ ]:
X_train = np.array(train_image)
y_train = np.array(train_labels)

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
train_dataset

In [ ]:
model = Sequential([
  layers.Rescaling(1./255, input_shape=(28, 28, 1)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(2)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
  train_dataset,
  epochs=epochs
)

# Build model

In [ ]:
df_joined = df_train.merge(hobbies, how='inner').query('(age >= age_low) & (age <= age_high) & (celebrity == celebrity)')
df_joined.drop(['age_low','age_high'],axis=1, inplace=True)
df_joined.head()

# Train images

In [ ]:
model.fit(train_dataset, epochs=10)

In [ ]:
df_test = pd.read_csv('/kaggle/input/kaggledays-flying-circus/test.csv')
df_test.head()

# Create merged dataset of hobbies and training set